In [2]:
import polars as pl
import polars.selectors as cs

import altair as alt
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [1]:
df_path = r'/Users/zygimantas/Documents/Data_sets/steamcharts.csv'

In [4]:
df = pl.read_csv(df_path, infer_schema_length=10000)

In [7]:
df.collect_schema()

Schema([('month', String),
        ('avg_players', Float64),
        ('gain', String),
        ('gain_percent', Float64),
        ('peak_players', Int64),
        ('name', String),
        ('steam_appid', Int64)])

In [8]:
df.columns

['month',
 'avg_players',
 'gain',
 'gain_percent',
 'peak_players',
 'name',
 'steam_appid']

In [9]:
print(df.head())

shape: (5, 7)
┌────────┬─────────────┬─────────┬──────────────┬──────────────┬────────────────┬─────────────┐
│ month  ┆ avg_players ┆ gain    ┆ gain_percent ┆ peak_players ┆ name           ┆ steam_appid │
│ ---    ┆ ---         ┆ ---     ┆ ---          ┆ ---          ┆ ---            ┆ ---         │
│ str    ┆ f64         ┆ str     ┆ f64          ┆ i64          ┆ str            ┆ i64         │
╞════════╪═════════════╪═════════╪══════════════╪══════════════╪════════════════╪═════════════╡
│ Sep-25 ┆ 7805.25     ┆ 883.12  ┆ 0.1276       ┆ 13254        ┆ Counter-Strike ┆ 10          │
│ Aug-25 ┆ 6922.13     ┆ -449.35 ┆ -0.061       ┆ 12168        ┆ Counter-Strike ┆ 10          │
│ Jul-25 ┆ 7371.48     ┆ -833.5  ┆ -0.1016      ┆ 13951        ┆ Counter-Strike ┆ 10          │
│ Jun-25 ┆ 8204.98     ┆ -847.53 ┆ -0.0936      ┆ 15798        ┆ Counter-Strike ┆ 10          │
│ May-25 ┆ 9052.51     ┆ -471.31 ┆ -0.0495      ┆ 15333        ┆ Counter-Strike ┆ 10          │
└────────┴─────────────┴──

In [17]:
df = df.with_columns(
    pl.col('gain').str.replace('-', '0').cast(pl.Float64),
    (pl.lit('2025 ') + pl.col('month')).alias('month').str.strptime(pl.Date(), '%Y %b-%d')
)

In [18]:
df.columns

['month',
 'avg_players',
 'gain',
 'gain_percent',
 'peak_players',
 'name',
 'steam_appid']

In [21]:
df = df.rename({
    'month': 'data'
})

In [22]:
df

data,avg_players,gain,gain_percent,peak_players,name,steam_appid
date,f64,f64,f64,i64,str,i64
2025-09-25,7805.25,883.12,0.1276,13254,"""Counter-Strike""",10
2025-08-25,6922.13,449.35,-0.061,12168,"""Counter-Strike""",10
2025-07-25,7371.48,833.5,-0.1016,13951,"""Counter-Strike""",10
2025-06-25,8204.98,847.53,-0.0936,15798,"""Counter-Strike""",10
2025-05-25,9052.51,471.31,-0.0495,15333,"""Counter-Strike""",10
…,…,…,…,…,…,…
2025-04-25,2.48,0.92,-0.2709,8,"""The Ditzy Demons Are in Love W…",802870
2025-03-25,3.4,0.19,-0.0532,11,"""The Ditzy Demons Are in Love W…",802870
2025-02-25,3.59,0.65,-0.1527,12,"""The Ditzy Demons Are in Love W…",802870
